In [176]:
import requests
import pandas as pd
import re

url = "https://raw.githubusercontent.com/Lorenzo1208/Projet_TrouveTonJob/main/data.json"
response = requests.get(url)
data = response.json()
df = pd.DataFrame(data)

# df['salaire_minimum'] = df['lieu'].apply(lambda x: x[1] if len(x) > 1 else '')
# df["salaire_maximum"] = df["salaire_minimum"].apply(lambda x: x.split("-")[1].strip() if '-' in x else '')
# df["salaire_minimum"] = df["salaire_minimum"].apply(lambda x: x.split("-")[0].strip() if '-' in x else x)

###########################################################
# Colonne [["salaire_minimum", "salaire_maximum"]]
###########################################################

# Divisez la colonne "lieu" en colonnes "salaire_minimum" et "salaire_maximum".
df[["salaire_minimum", "salaire_maximum"]] = df["lieu"].apply(lambda x: pd.Series(x[1].split("-")) if len(x) > 1 and '-' in x[1] else pd.Series(['', '']))

# Supprimer certains caractères et en remplacer d'autres dans la colonne "salaire_minimum".
df[["salaire_minimum", "salaire_maximum"]] = df[["salaire_minimum", "salaire_maximum"]].apply(lambda x: x.str.strip().str.replace(',','.').str.replace('€','').str.replace(' ','').str.replace('/an',''))

# Remplacer "selonprofil" par "50000" dans la colonne "salaire_maximum".
df["salaire_maximum"] = df["salaire_maximum"].str.replace('selonprofil', '50000')

# Remplacer ".000" par "000" dans les colonnes
df[["salaire_minimum", "salaire_maximum"]] = df[["salaire_minimum", "salaire_maximum"]].replace('.000', '000', regex=True)

###########################################################
# Colonne ["lieu"]
###########################################################
# Extraire le premier élément de la colonne "lieu" et le réaffecter au DataFrame.
df["lieu"] = df["lieu"].apply(lambda x: x[0])

###########################################################
# Toutes les colonnes 
###########################################################
# Supprimez les espaces et mettre en minuscules tous les éléments de toutes les colonnes, puis join avec des virgules.
df = df.apply(lambda c : c.apply(lambda x : ','.join([s.strip("\n ").lower() for s in (x.split(',') if type(x) != list else x)])))

###########################################################
# Colonne ["Intitulé du poste"]
###########################################################
# Enlever "(h/f)" de la colonne "Intitulé du poste".
df["Intitulé du poste"] = df["Intitulé du poste"].apply(lambda x: x.strip("(h/f)"))

###########################################################
# Colonne [["salaire_minimum", "salaire_maximum"]]
###########################################################
# Convert les éléments de la colonne "salaire_minimum" et "salaire_maximum" en float.
df["salaire_minimum"] = df["salaire_minimum"].apply(lambda x: float(x) if x else x)
df["salaire_maximum"] = df["salaire_maximum"].apply(lambda x: float(x) if x else x)

# Remplacez toutes les valeurs 0 dans la colonne "salaire_minimum" par 30000.
df["salaire_minimum"] = df["salaire_minimum"].replace(0,30000)
df["salaire_maximum"] = df["salaire_maximum"].replace(0,50000)

# Convertissez les éléments de la colonne "salaire_minimum" et "salaire_maximum" en nombres entiers et réaffectez-les au DataFrame.
df["salaire_minimum"] = df["salaire_minimum"].apply(lambda x: int(x) if x else None)
df["salaire_maximum"] = df["salaire_maximum"].apply(lambda x: int(x) if x else None)

df.to_csv("data_test1.csv", index=False)

In [173]:
df.dtypes

Intitulé du poste       object
Date de publication     object
lieu                    object
competences             object
Type de poste           object
salaire_minimum        float64
salaire_maximum        float64
dtype: object